# Decision Trees
Decision Trees (DTs) are a non-parametric supervised learning method used for classification and regression. The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features.

Decision trees are a relatively simple machine learning methode. It will run through different leaves, where it evaluates some condition. Depending on whether the condition is True or False for the instance we want to classify, it will go down either of two branches. There it will encounter a new node, until it has classified the instance.<br>

We'll use the iris dataset to get familiar with this method. `Scikit-learn` provides this dataset in its `datasets` subpackage. It will be loaded by the first function in the provided start material.

### Excercise
Try to make a decision tree to classify the instances of the iris dataset with this starting material. To use `tree.DecisionTreeClassifier()` import `tree` from `sklearn`. The classifier has the same syntax as the gaussian bayes classifier sklearn provides.

In [ ]:
# %load ./5_Advanced/tree.py
from sklearn import tree
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.67)

clf = tree.DecisionTreeClassifier(random_state=42)
clf = clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)

print('Accuracy tree = {0}'.format(accuracy))

## Visualizing the Decision Tree

In [ ]:
import graphviz
dot_data = tree.export_graphviz(clf, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("iris")
graph

## Cross-Validation

When evaluating different settings (“hyperparameters”) for estimators, there is still a risk of overfitting on the test set because the parameters can be tweaked until the estimator performs optimally. This way, knowledge about the test set can “leak” into the model and evaluation metrics no longer report on generalization performance. To solve this problem, yet another part of the dataset can be held out as a so-called “validation set”: training proceeds on the training set, after which evaluation is done on the validation set, and when the experiment seems to be successful, final evaluation can be done on the test set.

However, by partitioning the available data into three sets, we drastically reduce the number of samples which can be used for learning the model, and the results can depend on a particular random choice for the pair of (train, validation) sets.

A solution to this problem is a procedure called cross-validation (CV for short). A test set should still be held out for final evaluation, but the validation set is no longer needed when doing CV. In the basic approach, called k-fold CV, the training set is split into k smaller sets (other approaches are described below, but generally follow the same principles). The following procedure is followed for each of the k “folds”:
A model is trained using k-1 of the folds as training data;
the resulting model is validated on the remaining part of the data (i.e., it is used as a test set to compute a performance measure such as accuracy).
The performance measure reported by k-fold cross-validation is then the average of the values computed in the loop. This approach can be computationally expensive, but does not waste too much data (as it is the case when fixing an arbitrary test set), which is a major advantage in problem such as inverse inference where the number of samples is very small.

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, iris.data, iris.target, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

## Hyper Parameter Tuning

Tuning the hyper-parameters of a machine learning model is often carried out using an exhaustive exploration of (a subset of) the space all hyper-parameter configurations (e.g., using sklearn.model_selection.GridSearchCV), which often results in a very time consuming operation. 

In [ ]:
from sklearn.model_selection import GridSearchCV

criterion = ['gini','entropy']
max_depth = [4, None]

# Create the random grid
search_grid = {'criterion': criterion,
               'max_depth': max_depth}

In [ ]:
tree_grid = GridSearchCV(estimator = clf, 
                         param_grid = search_grid, 
                         cv = 3, verbose=1)
tree_grid.fit(X,y)

In [ ]:
print("Best score: %0.3f" % tree_grid.best_score_)
print("Best parameters set:")
print(tree_grid.best_params_)

## Ensemble Methods

Ensemble methods are techniques that create multiple models and then combine them to produce improved results.

Ensemble methods usually produces more accurate solutions than a single model would. This has been the case in a number of machine learning competitions, where the winning solutions used ensemble methods.  

It is important that we understand a few terminologies before we continue. Throughout this traing I used the term “model” to describe the output of the algorithm that trained with data. This model is then used for making predictions. This algorithm can be any machine learning algorithm such as logistic regression, decision tree, etc. These models, when used as inputs of ensemble methods, are called ”base models”.

### Majority Voting

Every model makes a prediction (votes) for each test instance and the final output prediction is the one that receives more than half of the votes. If none of the predictions get more than half of the votes, we may say that the ensemble method could not make a stable prediction for this instance. Although this is a widely used technique, you may try the most voted prediction (even if that is less than half of the votes) as the final prediction. In some articles, you may see this method being called “plurality voting”.

### Weighted Voting

Unlike majority voting, where each model has the same rights, we can increase the importance of one or more models. In weighted voting you give more weight to the prediction of the better models.

### Simple Averaging

In simple averaging method, for every instance of test dataset, the average predictions are calculated. This method often reduces overfit and creates a smoother regression model.

### Weighted Averaging

Weighted averaging is a slightly modified version of simple averaging, where the prediction of each model is multiplied by the weight and then their average is calculated.

### Bootstrapping
A problem with machine learning is that we only have a part of the data to train our algorithm with. Even if we wouldn't split the data between training and test data, we'll never have all the data about this problem in the real world. To get a model of the inference about a population from sample data, we can resample the sample data, allowing for replacement. Usually the resample size will be the same as the sample size, but they won't all be unique any more. By repeating this process 1.000 - 10.000 times we can make a model of the distribution of the statistic or estimator we want. (mean, standard variation, etc.) We now assume that the original sample has the same distribution towards the total population and we can model the inference.

### Bootstrap Aggregating or Bagging

The name Bootstrap Aggregating, also known as “Bagging”, summarizes the key elements of this strategy. In the bagging algorithm, the first step involves creating multiple models. These models are generated using the same algorithm with random sub-samples of the dataset which are drawn from the original dataset randomly with bootstrap sampling method. In bootstrap sampling, some original examples appear more than once and some original examples are not present in the sample. If you want to create a sub-dataset with m elements, you should select a random element from the original dataset m times. And if the goal is generating n dataset, you follow this step n times.

The second step in bagging is aggregating the generated models. Well known methods, such as voting and averaging, are used for this purpose.

In bagging, each sub-samples can be generated independently from each other. So generation and training can be done in parallel.

You can also find implementation of the bagging strategy in some algorithms. For example, Random Forest algorithm uses the bagging technique with some differences. Random Forest uses random feature selection, and the base algorithm of it is a decision tree algorithm (see below).

### Boosting: Converting Weak Models to Strong Ones

The term “boosting” is used to describe a family of algorithms which are able to convert weak models to strong models. The model is weak if it has a substantial error rate, but the performance is not random (resulting in an error rate of 0.5 for binary classification). Boosting incrementally builds an ensemble by training each model with the same dataset but where the weights of instances are adjusted according to the error of the last prediction. The main idea is forcing the models to focus on the instances which are hard. Unlike bagging, boosting is a sequential method, and so you can not use parallel operations here.

Adaboost is a widely known algorithm which is a boosting method. The founders of Adaboost won the Gödel Prize for their work. Mostly, decision tree algorithm is preferred as a base algorithm for Adaboost and in sklearn library the default base algorithm for Adaboost is decision tree (AdaBoostRegressor and AdaBoostClassifier).

## Example 1: Random Forest
A common way to try and improve the accuracy of a decision tree is the `sklearn.ensemble.RandomForestClassifier()`. This classifier will grow a number of fully grown decision trees from a bootstrapped sample and use them in parallel. The trees also only use a randomly selected subset of features for each decision. The fully grown decision trees will have a low bias, but a high variance. Since every tree has seen another set of data you can use them together and take an average of their outcomes to predict the target. (Each tree will 'vote' on a classification.) This will prevent overfitting, decrease the variance and improve the predictive accuracy. They are however also slower, because they use multiple decision trees and aren't guarenteed to be more accurate.

### Excercise
Try to implement a forest of 100 trees next to your tree and compare the results. use the `nb_estimators` parameter when initiating your forest. It might be usefull to run this a few times to get a good sense of the accuracy difference. 

In [ ]:
# %load 5_Advanced/forest.py
from sklearn import tree, ensemble
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.67)

clf = tree.DecisionTreeClassifier(random_state=42)
clf = clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
print('Accuracy tree = {0}%'.format(accuracy))
clf_rf = ensemble.RandomForestClassifier(n_estimators=100)
clf_rf.fit(X_train, y_train)
accuracy_rf = clf_rf.score(X_test, y_test)
print('Accuracy random forest = {0}%'.format(accuracy_rf))

## Example 2: Bagging & Boosting
Bagging and boosting are two **'meta-algorithms'** that seek to **improve the accuracy of the estimators**.
### Bagging
Bagging (__B__ootstrap __agg__regat__ing__) is very similar to Random Forest. The biggest difference is that it won't use a randomly selected subset of features for each decision. Instead it will use the **complete bootstrapped sample**, a random sample with replacement. This will generate trees with less diversity than Random Forest.

It uses a **majority vote** from classifiers trained on bootstrap samples of the training data.

#### Bagging Classifier
A Bagging classifier is an ensemble meta-estimator that **fits base classifiers** each on **random subsets of the original dataset** and then **aggregate their individual predictions (either by voting or by averaging) to form a final prediction**. Such a meta-estimator can typically be used as a way to **reduce the variance of a black-box estimator** (e.g., a decision tree), by introducing randomization into its construction procedure and then making an ensemble out of it.

#### Excercise
Use a `sklearn.ensemble.BaggingClassifier()` to make a model of the iris classification problem. Use the `KNeighborsClassifier()` as the base classifier and set `max_samples` to 0.5 and `max_features` also to 0.5.

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html

In [ ]:
# %load 5_Advanced/baggingClassification.py
import pandas as pd
import numpy as np
from sklearn import ensemble
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.67)

clf = ensemble.BaggingClassifier(random_state=42)
clf.fit(X_train, y_train)
accuracy  = clf.score(X_test, y_test)

print('Accuracy tree = {0}'.format(accuracy))

#### A Bagging Regressor
A Bagging regressor is an ensemble meta-estimator that **fits base regressors** each **on random subsets of the original dataset** and then **aggregate their individual predictions (either by voting or by averaging) to form a final prediction**. Such a meta-estimator can typically be used as a way to **reduce the variance of a black-box estimator** (e.g., a decision tree), by introducing randomization into its construction procedure and then making an ensemble out of it.

#### Excercise
Make an estimation of the price of houses in boston with a `sklearn.ensemble.BaggingRegressor()`.

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html

In [ ]:
%load 5_Advanced/baggingRegression.py


#### AdaBoost
Boosting will use a number of **weak learners**. If were talking about decision trees weak learners would be small trees, with maybe only one or two nodes. If these weak learners are better than random guesses, they can be **combined into one strong learner**. These weak learners have a **high bias** and a **low variance**, so the boosting algorithm will mainly aim to **reduce the bias**.

AdaBoost is a **popular boosting algorithm**. It will fit a sequence of weak learners **on repeatedly modified versions of the data**. Their predictions are then combined in a **weighted majority vote or sum**. The data that is training the AdaBoost algorithm will also receive weights. **At each step those samples that weren't predicted correctly will gain weight, while the correct samples' weight will decrease**. This way the weak learners have to focus on the difficult samples.

##### AdaBoost Classifier
An AdaBoost Classifier is a meta-estimator that **begins by fitting a classifier on the original dataset** and then **fits additional copies of the classifier on the same dataset but where the weights of incorrectly classified instances are adjusted** such that subsequent classifiers focus more on difficult cases.

##### AdaBoost Regressor
An AdaBoost regressor is a meta-estimator that **begins by fitting a regressor on the original dataset** and then **fits additional copies of the regressor on the same dataset but where the weights of instances are adjusted according to the error of the current prediction.** As such, subsequent regressors focus more on difficult cases.
This class implements the algorithm known as **AdaBoost.R2**.

#### Excercise
Use the `sklearn.ensemble.AdaBoostClassifier()` and the `sklearn.ensemble.AdaBoostRegressor()` on the iris and boston dataset respectively.

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html

In [ ]:
%load 5_Advanced/adaboost.py


## Example 3: Gradient Boosting
Gradient boost is based on the idea that **boosting weak learners into a strong one, can be thought of as minimizing a arbitrary differentiable 'loss' function, the difference between our data and our predictions**. It's important that this loss function be differentiable, because we can minimize a function by using the gradient descent method, which uses the gradient as a reference for how far you are from the minimum. At a minimum the gradient becomes 0.<br>
Gradient boost uses these principles to combine weak learners.

### Gradient Boosting for Classification
GB **builds an additive model in a forward stage-wise fashion**; it allows for the optimization of arbitrary differentiable loss functions. In each stage n_classes_ regression trees are fit on the negative gradient of the binomial or multinomial deviance loss function. Binary classification is a special case where only a single regression tree is induced.

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

### Gradient Boosting for Regression
GB **builds an additive model in a forward stage-wise fashion**; it allows for the optimization of arbitrary differentiable loss functions. In each stage a regression tree is fit on the negative gradient of the given loss function.

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html

#### Excercise
Use the `sklearn.ensemble.GradientBoostingClassifier()` and the `sklearn.ensemble.GradientBoostingRegressor()` on the iris and boston dataset respectively.

In [ ]:
%load 5_Advanced/gradientboost.py
